# Policy Evaluation

In [3]:
import numpy as np

## Gridworld Example

In [20]:
def get_pos(ix, lenght):
    """
    Get the position in a square Gridworld
    """
    col = ix % length
    row = ix // length
    state = np.asarray([row, col])
    return state